## Personas of Thought
Asking the LLM to generate a crowd of personas to answer a question before aggregating their feedback.

In [1]:
from openai import OpenAI

question = "I'm starting a company that makes shoes that fit any foot size. What do you think of the name 'OmniFit'?"
number = 10

def call_openai(messages):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

system_prompt = "You are a helpful and terse assistant."
question_prompt = "I want a paragraph response to the following question: {question}"

naive_messages = [
    {"role": "system", "content": system_prompt}, 
    {"role": "user", "content": question_prompt.format(question=question)}
]

naive_response = call_openai(naive_messages)

print(naive_response)


The name 'OmniFit' is a strong choice for your shoe company. It conveys a sense of inclusivity and versatility, suggesting that your shoes are designed to accommodate all foot sizes. The prefix "Omni" implies universality, which aligns perfectly with your mission to provide a fit for everyone. Additionally, the name is catchy and easy to remember, making it effective for branding and marketing. Overall, 'OmniFit' effectively communicates your product’s unique selling proposition.


In [4]:
experts_prompt = """I want a paragraph response to the following question: {question}

First, name {number} world-class experts (past or present) who would be great at answering this?

Then for each expert, please answer the question critically from their perspective given their background and experience.

Finally, combine the responses into a single response as if these experts had collaborated in writing a joint anonymous answer.

## Expert names:
Expert name (relevant experience)
Expert name (relevant experience)
Expert name (relevant experience)
...

### Expert responses:
Expert name: <response>
Expert name: <response>
Expert name: <response>
...

### Final response:
<joint response>
"""

experts_messages = [
    {"role": "system", "content": system_prompt}, 
    {"role": "user", "content": experts_prompt.format(question=question, number=number)}
]

experts_response = call_openai(experts_messages)

print(experts_response)

### Expert names:
1. Tinker Hatfield (Footwear Designer, Nike)
2. Dr. Robert H. Grubbs (Nobel Laureate in Chemistry, Material Science)
3. Dr. David K. Cavanagh (Biomechanist, Footwear Research)
4. Dr. Martine Rothblatt (Biotech Entrepreneur, Inventor)
5. Tim Brown (Co-founder, Allbirds)
6. Dr. Susan H. Wilson (Consumer Behavior Expert)
7. Steve Madden (Footwear Designer, Entrepreneur)
8. Dr. Andrew J. M. Smith (Orthopedic Surgeon, Foot Specialist)
9. Dr. Jennifer L. A. McMahon (Ergonomics Expert)
10. Howard Schultz (Former CEO, Starbucks, Branding Expert)

### Expert responses:
**Tinker Hatfield:** The name 'OmniFit' is strong and suggests versatility, which is essential for a shoe that accommodates any foot size. It embodies innovation in footwear design.

**Dr. Robert H. Grubbs:** From a materials science perspective, the name implies the possibility of a universal fit, which is intriguing. It would be vital to ensure that the materials used truly deliver on this promise.

**Dr. Davi

In [5]:
personas_prompt = """I want a paragraph response to the following question: {question}

First, name {number} demographic personas who would be relevant for answering this?

Then for each persona, please answer the question critically from their perspective given their background and experience.

Finally, combine the responses into a single response as if these personas had collaborated in writing a joint anonymous answer.

### Persona names:
Persona name (relevant demographics)
Persona name (relevant demographics)
Persona name (relevant demographics)
...

### Persona responses:
Persona name: <response>
Persona name: <response>
Persona name: <response>
...

### Final response:
<joint response>
"""

personas_messages = [
    {"role": "system", "content": system_prompt}, 
    {"role": "user", "content": personas_prompt.format(question=question, number=number)}
]

personas_response = call_openai(personas_messages)

print(personas_response)

### Persona names:
1. Athletic Coach (sports enthusiasts, active lifestyle)
2. Footwear Retail Manager (retail experience, consumer insights)
3. Fashion Blogger (style-conscious individuals, trendsetters)
4. Physical Therapist (healthcare, injury prevention)
5. Parent of Children with Special Needs (focus on accessibility, adaptability)
6. Podologist (foot specialists, health professionals)
7. Technology Entrepreneur (innovation, tech-driven solutions)
8. Environmental Activist (sustainability, eco-conscious consumers)
9. Senior Citizen (age-related foot issues, comfort priorities)
10. College Student (budget-conscious, trend-aware)

### Persona responses:
Athletic Coach: “'OmniFit' is a strong name that conveys inclusivity and adaptability, appealing to athletes who come in all shapes and sizes. However, it may lack the boldness typically associated with performance footwear.”

Footwear Retail Manager: “The name 'OmniFit' captures the essence of versatility, which is essential in toda

In [6]:
def extract_final_response(response):
    return response.split("### Final response:")[1].strip()

experts_final_response = extract_final_response(experts_response)
personas_final_response = extract_final_response(personas_response)

print("Naive: ", naive_response)
print("-"*100)
print("Experts: ", experts_final_response)
print("-"*100)
print("Personas: ", personas_final_response)


Naive:  The name 'OmniFit' is a strong choice for your shoe company. It conveys a sense of inclusivity and versatility, suggesting that your shoes are designed to accommodate all foot sizes. The prefix "Omni" implies universality, which aligns perfectly with your mission to provide a fit for everyone. Additionally, the name is catchy and easy to remember, making it effective for branding and marketing. Overall, 'OmniFit' effectively communicates your product’s unique selling proposition.
----------------------------------------------------------------------------------------------------
Experts:  The name 'OmniFit' is compelling and effectively conveys the idea of inclusivity and adaptability in footwear, appealing to a broad consumer base. It suggests a commitment to innovation and versatility, essential qualities for a product that aims to accommodate any foot size. However, to ensure credibility, it is crucial that the product is supported by robust research and high-quality materia

In [7]:
judge_prompt = """I want you to compare two responses to the following question: {question}

Here are the two responses:

Response 1:
{response1}

Response 2: 
{response2}

Please analyze both responses and choose which one is better. Consider factors like:
- Uniqueness of perspectives
- Nonobvious insights
- Sounds human not like an AI

## Your analysis:
<analysis>

## Your choice:
The better response is: Response <1 or 2>
"""

def judge_responses(response1, response2):
    judge_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": judge_prompt.format(
            question="How can I be more productive?",
            response1=response1,
            response2=response2
        )}
    ]

    judge_response = call_openai(judge_messages)
    
    choice = judge_response.split("The better response is: Response ")[1].strip()
    return {"choice": int(choice), "reason": judge_response}


naive_vs_experts = judge_responses(naive_response, experts_final_response)
print("# Naive vs experts:", naive_vs_experts["choice"])

naive_vs_personas = judge_responses(naive_response, personas_final_response)
print("# Naive vs personas:", naive_vs_personas["choice"])

experts_vs_personas = judge_responses(experts_final_response, personas_final_response)
print("# Experts vs personas:", experts_vs_personas["choice"])

print("-"*100)
print(naive_vs_experts)
print("-"*100)
print(naive_vs_personas)
print("-"*100)
print(experts_vs_personas)

# Naive vs experts: 2
# Naive vs personas: 2
# Experts vs personas: 1
----------------------------------------------------------------------------------------------------
{'choice': 2, 'reason': "## Your analysis:\nResponse 1 provides a clear and concise endorsement of the name 'OmniFit', emphasizing its strengths in terms of inclusivity, memorability, and branding. However, its insights are somewhat surface-level and lack depth regarding practical execution or market strategy.\n\nResponse 2, on the other hand, offers a more nuanced perspective. It not only acknowledges the name's strengths but also highlights the importance of backing the branding with research, quality, and strategic marketing. This response delves deeper into the implications of the name and considers the competitive landscape, providing a more comprehensive view of what will make 'OmniFit' successful.\n\nResponse 2 also sounds more human due to its acknowledgment of the need for execution and connection with consum

### Bringing it all together

In [8]:
questions = [
    "What is the best way to learn a new skill?",
    "What is the best way to stay healthy?",
    "How can I be more productive?",
    "What will AI look like in 10 years?",
    "How do we end world hunger?",
]

number = 10

def run_test(question):

    naive_messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": question_prompt.format(question=question)}
    ]

    naive_response = call_openai(naive_messages)

    print(naive_response)


    experts_messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": experts_prompt.format(question=question, number=number)}
    ]

    experts_response = call_openai(experts_messages)

    print(experts_response)


    personas_messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": personas_prompt.format(question=question, number=number)}
    ]

    personas_response = call_openai(personas_messages)

    print(personas_response)


    experts_final_response = extract_final_response(experts_response)
    personas_final_response = extract_final_response(personas_response)

    print("Naive: ", naive_response)
    print("-"*100)
    print("Experts: ", experts_final_response)
    print("-"*100)
    print("Personas: ", personas_final_response)

    naive_vs_experts = judge_responses(naive_response, experts_final_response)
    print("# Naive vs experts:", naive_vs_experts["choice"])

    naive_vs_personas = judge_responses(naive_response, personas_final_response)
    print("# Naive vs personas:", naive_vs_personas["choice"])

    experts_vs_personas = judge_responses(experts_final_response, personas_final_response)
    print("# Experts vs personas:", experts_vs_personas["choice"])

    print("-"*100)
    print(naive_vs_experts)
    print("-"*100)
    print(naive_vs_personas)
    print("-"*100)
    print(experts_vs_personas)

    return {
        "naive": naive_response,
        "experts": experts_final_response,
        "personas": personas_final_response,
        "naive_vs_experts": naive_vs_experts,
        "naive_vs_personas": naive_vs_personas,
        "experts_vs_personas": experts_vs_personas,
    }

results = {}

for question in questions:
    results[question] = run_test(question)


The best way to learn a new skill is through a combination of structured practice, active engagement, and feedback. Start by breaking the skill down into manageable components and setting clear, achievable goals. Utilize a variety of learning resources, such as tutorials, books, or online courses, to gain foundational knowledge. Consistent practice is crucial, so dedicate regular time to apply what you've learned in real-life scenarios. Seek feedback from knowledgeable sources to identify areas for improvement and adjust your approach accordingly. Finally, maintain a growth mindset—embrace challenges and view mistakes as opportunities to enhance your understanding and proficiency.
### Expert names:
1. Malcolm Gladwell (author and journalist)
2. Angela Duckworth (psychologist and author of "Grit")
3. Tony Robbins (motivational speaker and life coach)
4. Joshua Waitzkin (chess master and martial artist)
5. Carol Dweck (psychologist known for mindset research)
6. Tim Ferriss (entrepreneur

In [9]:
# Count wins for each approach
wins = {"naive": 0, "experts": 0, "personas": 0}
trials = {"naive": 0, "experts": 0, "personas": 0}

for question in results:
    # Naive vs experts
    if results[question]["naive_vs_experts"]["choice"] == 1:
        wins["naive"] += 1
    else:
        wins["experts"] += 1
    trials["naive"] += 1
    trials["experts"] += 1
    
    # Naive vs personas 
    if results[question]["naive_vs_personas"]["choice"] == 1:
        wins["naive"] += 1
    else:
        wins["personas"] += 1
    trials["naive"] += 1
    trials["personas"] += 1
    
    # Experts vs personas
    if results[question]["experts_vs_personas"]["choice"] == 1:
        wins["experts"] += 1
    else:
        wins["personas"] += 1
    trials["experts"] += 1
    trials["personas"] += 1

# Calculate percentages
percentages = {
    approach: (wins[approach] / trials[approach]) * 100 
    for approach in wins
}

print("\nWin percentages:")
for approach, percentage in percentages.items():
    print(f"{approach}: {percentage:.1f}%")



Win percentages:
naive: 10.0%
experts: 60.0%
personas: 80.0%
